# Importation des modules

In [1]:
# Pour manipuler et visualiser les données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pour séparer et évaluer les données
from sklearn.model_selection import train_test_split, cross_validate, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

# Pour préparer les données
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# Pour créer des arbres de classification
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

# Pour faire un modèle de régression logistique 
from sklearn.linear_model import LogisticRegression

# Pour utiliser xgboost
import xgboost as xgb

# Pour utiliser les métriques
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Pour exporter notre modèle
import pickle5 as pickl 

# Création du dataset et Dummy Classifier

On commence par créer un dataset avec les variables :
<p style='color: #FFA07A'> NAICS, NoEmp, RetainedJob, UrbanRural, Term, UrbanRural, MIS_Status </p>

In [2]:
df = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_112567/3668100114.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/SBAnational_clean.csv")


In [3]:
""" df['NAICS'] = df['NAICS'].astype(str).apply(lambda x : x[:2])
df[['NAICS']] = df[['NAICS']].astype(int) """

" df['NAICS'] = df['NAICS'].astype(str).apply(lambda x : x[:2])\ndf[['NAICS']] = df[['NAICS']].astype(int) "

In [4]:
# Dataframe contenant les variables explicatives
X = df[['NAICS', 'NoEmp',  'RetainedJob', 'UrbanRural','Term']]

# On transforme les valeurs de MIS_Status en 0 ou 1
y = df['MIS_Status'].astype('category').cat.codes

On crée maintenant notre jeu de données d'entraînement et de test :

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42, stratify=y)

On sépare les variables numériques et catégorielle :

In [7]:
var_num = ['NoEmp','RetainedJob','Term',"NAICS"]

var_cat = [ 'UrbanRural']

Puis on crée un transformateur de colonne :

In [8]:
preprocessor = make_column_transformer(
    (StandardScaler(), var_num),
    (OneHotEncoder(), var_cat)
)

On commence notre modélisation par un Dummy Classifier qui servira de point de comparaison :

In [9]:
dummy = make_pipeline(preprocessor, DummyClassifier())

dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

dummy_report = (classification_report(y_test, y_pred))
print(dummy_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     31195
           1       0.82      1.00      0.90    145830

    accuracy                           0.82    177025
   macro avg       0.41      0.50      0.45    177025
weighted avg       0.68      0.82      0.74    177025



/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

# Random Forest Classifier

Nous allons maintenant utiliser un Random Forest Classififier sans paramètres pour le comparer avec notre Dummy Classifier :

In [10]:
# Entraînement du modèle
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier())
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NoEmp', 'RetainedJob',
                                                   'Term', 'NAICS']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['UrbanRural'])])),
                ('randomforestclassifier', RandomForestClassifier())])

On affiche l'importance des features :

In [11]:
importances = rf_pipe[-1].feature_importances_
importances
# features = pd.DataFrame({"feature": X_train.columns, "importance": importances})
# features

array([0.05206518, 0.04423988, 0.68644431, 0.1848851 , 0.01921203,
       0.01070674, 0.00244676])

In [12]:
rf_pipe.get_feature_names_out

<bound method Pipeline.get_feature_names_out of Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NoEmp', 'RetainedJob',
                                                   'Term', 'NAICS']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['UrbanRural'])])),
                ('randomforestclassifier', RandomForestClassifier())])>

In [13]:
y_pred = rf_pipe.predict(X_test)

rfc_report = classification_report(y_test, y_pred)
print(rfc_report)

              precision    recall  f1-score   support

           0       0.78      0.72      0.75     31195
           1       0.94      0.96      0.95    145830

    accuracy                           0.92    177025
   macro avg       0.86      0.84      0.85    177025
weighted avg       0.91      0.92      0.91    177025



En utilisant un Random Forest Classifier sans paramètres, on obtient de bons résultats, proches de notre Dummy Classifier pour les classes positives. <br>
On va essayer d'affiner notre modèle avec en cherchant les meilleurs hyperparamètres :

In [17]:
""" # On définit les paramètres à rechercher avec un intervalle de recherche
param_dist = {'n_estimators': np.arange(95, 100, 1), 
              'min_samples_split': np.arange(1, 2),
              'min_samples_leaf': np.arange(1, 2),
              'criterion': ['gini']
}
# On crée un objet Random Search 
random_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,
                                  cv=5, n_jobs=-1, verbose=2)

# On entrâine le modèle
random_search.fit(X_train, y_train)

# On récupère les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_ """
best_params = {'n_estimators': 95, 'min_samples_split': 1, 'min_samples_leaf': 1, 'criterion': 'gini'}

# Entraînement du modèle
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier(**best_params, class_weight="balanced"))
rf_pipe.fit(X_train, y_train)

y_pred = rf_pipe.predict(X_test)

print(f"resultats après random search :\n {classification_report(y_test, y_pred)}")
print(f"resultats avant random search :\n {rfc_report}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 98, 'min_samples_split': 1, 'min_samples_leaf': 1, 'criterion': 'gini'}
resultats après random search :
               precision    recall  f1-score   support

           0       0.74      0.74      0.74     31195
           1       0.95      0.94      0.94    145830

    accuracy                           0.91    177025
   macro avg       0.84      0.84      0.84    177025
weighted avg       0.91      0.91      0.91    177025

resultats avant random search :
               precision    recall  f1-score   support

           0       0.78      0.72      0.75     31195
           1       0.94      0.96      0.95    145830

    accuracy                           0.92    177025
   macro avg       0.86      0.84      0.85    177025
weighted avg       0.91      0.92      0.91    177025



In [18]:
print(best_params)

{'n_estimators': 98, 'min_samples_split': 1, 'min_samples_leaf': 1, 'criterion': 'gini'}


In [ ]:
# copier ici le best params obtenu et definir best params ensuite 

# Régression logistique

In [19]:
# Entraînement du modèle
rf_pipe_log = make_pipeline(preprocessor, LogisticRegression())
rf_pipe_log.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NoEmp', 'RetainedJob',
                                                   'Term', 'NAICS']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['UrbanRural'])])),
                ('logisticregression', LogisticRegression())])

In [20]:
y_pred = rf_pipe_log.predict(X_test)

log_report = classification_report(y_test, y_pred)
print(log_report)

              precision    recall  f1-score   support

           0       0.65      0.18      0.29     31195
           1       0.85      0.98      0.91    145830

    accuracy                           0.84    177025
   macro avg       0.75      0.58      0.60    177025
weighted avg       0.81      0.84      0.80    177025



In [23]:
""" # #On définit les paramètres à rechercher avec un intervalle de recherche
param_dist = {'C': np.logspace(-4, 4, 20),
              'penalty': [ 'elasticnet',"l1","l2"],    #l1,l2
              'solver': ['newton-cg', 'lbfgs', ]}    #'sag', 'saga'

# On crée un objet Random Search 
random_search = RandomizedSearchCV(rf_pipe_log[-1], param_distributions=param_dist,
                                   n_iter=50, cv=3, n_jobs=-1, verbose=1)

# On entrâine le modèle
random_search.fit(X_train, y_train) 

# On récupère les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_
print(best_params) """
best_params = {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.0001}
# Entraînement du modèle
rf_pipe_log_opti = make_pipeline(preprocessor, LogisticRegression(**best_params, class_weight="balanced"))
rf_pipe_log_opti.fit(X_train, y_train)

y_pred = rf_pipe_log_opti.predict(X_test)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.0001}


/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [24]:
print(f"resultats après grid search :\n {classification_report(y_test, y_pred)}")
print(f"resultats avant grid search :\n {log_report}")

resultats après grid search :
               precision    recall  f1-score   support

           0       0.31      0.83      0.45     31195
           1       0.94      0.60      0.73    145830

    accuracy                           0.64    177025
   macro avg       0.63      0.71      0.59    177025
weighted avg       0.83      0.64      0.68    177025

resultats avant grid search :
               precision    recall  f1-score   support

           0       0.65      0.18      0.29     31195
           1       0.85      0.98      0.91    145830

    accuracy                           0.84    177025
   macro avg       0.75      0.58      0.60    177025
weighted avg       0.81      0.84      0.80    177025



In [25]:
print(best_params)

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.0001}


# XGBoost

In [26]:
# Entraînement du modèle
xgb_pipe = make_pipeline(preprocessor, xgb.XGBClassifier())
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NoEmp', 'RetainedJob',
                                                   'Term', 'NAICS']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['UrbanRural'])])),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stoppi...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [27]:
y_pred = xgb_pipe.predict(X_test)

xgb_report = classification_report(y_test, y_pred)
print(xgb_report)

              precision    recall  f1-score   support

           0       0.82      0.78      0.80     31195
           1       0.95      0.96      0.96    145830

    accuracy                           0.93    177025
   macro avg       0.89      0.87      0.88    177025
weighted avg       0.93      0.93      0.93    177025



In [31]:
""" param_dist = {'n_estimators': np.arange(95, 100, 5), 
              'learning_rate': np.linspace(0.01, 1, 20),
              'subsample': np.linspace(0.1, 1, 10),
              'colsample_bytree': np.linspace(0.2, 1, 10)
             }

random_search = RandomizedSearchCV(xgb_pipe[-1], param_distributions=param_dist,
                                   n_iter=50, cv=5, n_jobs=-1, verbose=1)

random_search.fit(X_train, y_train)

# On récupère les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_ """

#print(best_params)
best_params = {'subsample': 1.0, 'n_estimators': 95, 'learning_rate': 0.4789473684210527, 'colsample_bytree': 0.6444444444444445}

# Entraînement du modèle
rf_pipe_xgb_opti = make_pipeline(preprocessor, xgb.XGBClassifier(**best_params, class_weight="balanced"))
rf_pipe_log_opti.fit(X_train, y_train)

y_pred = rf_pipe_log_opti.predict(X_test)

[15:45:18] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.



In [29]:
print(best_params)

{'subsample': 1.0, 'n_estimators': 95, 'learning_rate': 0.4789473684210527, 'colsample_bytree': 0.6444444444444445}


In [30]:
print(f"resultats après grid search : {classification_report(y_test, y_pred)}")
print(f"resultats avant grid search : {xgb_report}")

resultats après grid search :               precision    recall  f1-score   support

           0       0.82      0.78      0.80     31195
           1       0.95      0.96      0.96    145830

    accuracy                           0.93    177025
   macro avg       0.89      0.87      0.88    177025
weighted avg       0.93      0.93      0.93    177025

resultats avant grid search :               precision    recall  f1-score   support

           0       0.82      0.78      0.80     31195
           1       0.95      0.96      0.96    145830

    accuracy                           0.93    177025
   macro avg       0.89      0.87      0.88    177025
weighted avg       0.93      0.93      0.93    177025



In [33]:
import pickle as pickle

In [35]:
pickle_out = open("modelrandomforest.pkl", "wb") 
pickle.dump(rf_pipe, pickle_out) 
pickle_out.close()
pickle_out4 = open("modelregressionlogistic.pkl", "wb") 
pickle.dump(rf_pipe_log_opti, pickle_out4) 
pickle_out4.close()
pickle_out5 = open("modelxgboost.pkl", "wb") 
pickle.dump(rf_pipe_xgb_opti, pickle_out5) 
pickle_out4.close()